In [7]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 18.0 MB/s eta 0:00:00m eta 0:00:010:01:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [33]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import xgboost as xgb
import pickle
from joblib import dump, load
from sklearn.metrics import precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import random
from transformers import set_seed
from scipy.special import softmax

In [25]:
test_data = pd.read_csv("test_df full.csv")
train_data = pd.read_csv("new_train_df (1).csv")
test_data.drop(columns = ["Unnamed: 0"], inplace=True)

In [5]:
train_data

,index,id1,id2,is_in_same_sent,min_sents_window,sentence,in_neighbors,label,gda_score,gda_disparities
0,1834320,3073,D013661,False,1,Biochemistry and genetics of @DiseaseTgt$ . @D...,0 0 0 4 2|4 0 6|8 8 8 13 13 13 13 8 16 16 13 8...,1,0.650000,2.0
1,10694284,3077,D006432,True,0,@GeneSrc$ @DiseaseTgt$ . @DiseaseTgt$ is a com...,1 1 1 3|8 8 8 8 8 8 12 12 12 8 8 15 16 16 19 1...,1,0.650000,2.0
2,11845294,1080,D003550,True,0,@DiseaseTgt$ and @GeneSrc$ . @DiseaseTgt$ ( @D...,0 0 0 0 4|11 6 4 6 11 11 11 11 11 15 15 12 11 ...,1,0.573333,3.0
3,12416729,3077,D006432,True,0,@GeneSrc$ and @GeneSrc$ @DiseaseTgt$ . @Diseas...,0 0 3 0 0 5|11 7 5 7 11 11 11 14 14 11 16 17 1...,0,0.650000,2.0
4,12416729,3077,D008107,False,1,@GeneSrc$ and @GeneSrc$ hemochromatosis . Here...,0 0 3 0 0 6 12 8 6 8 12 12 12 15 15 12 17 18 1...,0,0.100000,1.0
...,...,...,...,...,...,...,...,...,...,...
4536,37366140,9829,D010300,True,0,@DiseaseTgt$ - genetic cause . PURPOSE OF REVI...,0|2|3 2 3 3 3 5 7 5 5 10 19 14 14 14 10 14 15|...,1,0.050000,1.0
4537,37387251,4000,D016884,True,0,A recurrent homozygous @GeneSrc$ missense vari...,6 6 6 6 6 6 7 8 8 11 11 8 11 15 15 12 15 19 19...,1,0.060000,1.0
4538,37468280,4771,C537392,True,0,Current progress in genomics and targeted ther...,1 1 3 1 3 6 1 9 9 6 9 1 13 26 13 16 13 16 13 2...,1,0.200000,1.0
4539,37522971,3043,D000755,True,0,Recent national trends in outcomes and economi...,2 2 2 4 2 4 7 4 13 13 13 12 13 2 2 15|22 17 15...,1,0.800000,1.0


In [21]:

class model_trainer:

    def __init__(self, train_data, test_data, model_name):
        self.train_df= train_data
        self.test_df= test_data
        self.model_name=self.load_model(model_name)
        self.tokenizer=self.load_tokenizer(model_name)
#         self.model_name.resize_token_embeddings(len(self.tokenizer))
        

    def load_model(self,model_name):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        biobert = BertModel.from_pretrained(model_name).to(self.device)
        print("BioBERT model loaded")
        return biobert

    def load_tokenizer(self,model_name):
        tokenizer = BertTokenizer.from_pretrained(model_name)
#         tokenizer.add_tokens(["GeneSrc", "DiseaseTgt", "causative", "causal", "cause", "causing", "caused"])
        print("Tokenizer loaded")
        return tokenizer

    def load_dataset(self,data_path):
        df = pd.read_csv(data_path)
        return df
    
    def remove_stopwords(self,text):
        #nltk.download('stopwords')
        stop_words = set(stopwords.words('english'))
        
        tokens = nltk.word_tokenize(text)
        tokens = [word for word in tokens if word.lower() not in stop_words]
        return ' '.join(tokens)
    
    
    def create_dataset(self):
        # Apply the remove_stopwords function to the 'sentence' column
        # self.train_df['sentence'] = self.train_df['sentence'].apply(self.remove_stopwords)
        # self.test_df['sentence'] = self.test_df['sentence'].apply(self.remove_stopwords)

        X_train = self.train_df['sentence'].tolist()
        y_train = self.train_df['label'].tolist()

        X_test = self.test_df['sentence'].tolist()
        y_test = self.test_df['label'].tolist()
        
        print("Dataset created")
        # print(X_test[0])
        return X_train,y_train,X_test,y_test
    
    def get_specific_token_embeddings(self,sentence):
        # 1. Tokenize the input sentence
        inputs = self.tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512).to(self.device)

        # 2. Find the indices of "@GeneSrc" and "@DiseaseTgt$"
        tokenized_sentence = self.tokenizer.tokenize(sentence)
        gene_src_token = self.tokenizer.tokenize("@GeneSrc$")
        disease_tgt_token = self.tokenizer.tokenize("@DiseaseTgt$")

        gene_src_indices = [i for i, token in enumerate(tokenized_sentence) if token in gene_src_token]
        disease_tgt_indices = [i for i, token in enumerate(tokenized_sentence) if token in disease_tgt_token]

        # Run the sentence through BioBERT
        with torch.no_grad():
            #print(self.model_name)
            outputs = self.model_name(**inputs)
        embeddings = outputs['last_hidden_state'][0]  # Extracting embeddings for the whole sentence

        # 3. Retrieve the embeddings for the surrounding tokens
        context_range = 2

        def get_context_embeddings(indices):
            context_embeddings = []
            for idx in indices:
                start = max(0, idx - context_range)
                end = min(idx + context_range + 1, len(tokenized_sentence))
                context = embeddings[start:end]
                context_embeddings.append(context)
            return torch.cat(context_embeddings).view(-1, 768)

        gene_src_embeddings = get_context_embeddings(gene_src_indices)
        disease_tgt_embeddings = get_context_embeddings(disease_tgt_indices)

        # 4. Compute the average of the embeddings
        avg_gene_src_embedding = torch.mean(gene_src_embeddings, dim=0)
        avg_disease_tgt_embedding = torch.mean(disease_tgt_embeddings, dim=0)

        combined_embedding = torch.cat([avg_gene_src_embedding, avg_disease_tgt_embedding], dim=0)
        combined_embedding_np = combined_embedding.cpu().numpy().reshape(1, -1)  # Convert tensor to NumPy array and reshape to 2D

        if np.isnan(combined_embedding_np).any():
            print(sentence)

        return combined_embedding_np

        
        
    # def generate_embeddings(self):
    #     X_train,y_train,X_test,y_test=self.create_dataset()
    #     try:
    #         X_train_embeddings = np.vstack([self.get_specific_token_embeddings(sentence) for sentence in tqdm(X_train)])
    #         X_test_embeddings = np.vstack([self.get_specific_token_embeddings(sentence) for sentence in tqdm(X_test)])
           
    #     except Exception as e:
    #         print(e)
    #         pass
    #     print("Embeddings generated")
    #     return X_train_embeddings,X_test_embeddings

    def generate_embeddings(self):
        X_train, y_train, X_test, y_test = self.create_dataset()
        
        X_train_embeddings = []
        valid_train_indices = []
        X_test_embeddings = []
        valid_test_indices = []
        
        # Generate train embeddings
        for idx, sentence in enumerate(tqdm(X_train)):
            embedding = self.get_specific_token_embeddings(sentence)
            if embedding is not None:
                X_train_embeddings.append(embedding)
                valid_train_indices.append(idx)
        
        # Filter labels based on valid embeddings
        y_train = [y_train[idx] for idx in valid_train_indices]
        
        # Generate test embeddings
        for idx, sentence in enumerate(tqdm(X_test)):
            embedding = self.get_specific_token_embeddings(sentence)
            if embedding is not None:
                X_test_embeddings.append(embedding)
                valid_test_indices.append(idx)
        
        # Filter labels based on valid embeddings
        y_test = [y_test[idx] for idx in valid_test_indices]
        
        X_train_embeddings = np.vstack(X_train_embeddings)
        X_test_embeddings = np.vstack(X_test_embeddings)
        
        print("Embeddings generated")
        return X_train_embeddings, X_test_embeddings

    
    def drop_null_embeddings(self,X_train_embeddings,X_test_embeddings,y_train,y_test):
        X_train_embeddings=pd.DataFrame(X_train_embeddings).dropna()
        X_test_embeddings=pd.DataFrame(X_test_embeddings).dropna()

        test_ind=[i for i in range(0,len(X_test_embeddings)) if i not in X_test_embeddings.index]
        train_ind=[i for i in range(0,len(X_train_embeddings)) if i not in X_train_embeddings.index]
        for i in train_ind:
            y_train.pop(i)
        for i in test_ind:
            y_test.pop(i)
            
        return X_train_embeddings,X_test_embeddings,y_train,y_test
    
    def svm_classifiation(self, X_train_embeddings,y_train,X_test_embeddings,y_test):
        print("Doing classification using SVM...")
        clf = SVC(kernel='poly', degree=20, probability=True, class_weight={0:1, 1:30})
        clf.fit(X_train_embeddings, y_train)

        # Predict and Evaluate
        y_pred = clf.predict(X_test_embeddings)
        print("Results of SVM classifier are: ")
        print(classification_report(y_test, y_pred))
        probas = clf.predict_proba(X_test_embeddings)  # Get probabilities for the positive class
        y_scores = probas[:, 1]  # Probabilities for class 1
    
        # Example: Compute precision-recall curve
        precision, recall, _ = precision_recall_curve(y_test, y_scores)
        
        # Example: Compute AUC (Area Under Curve) for precision-recall curve
        auprc = auc(recall, precision)
        
        print(f'Area Under Precision-Recall Curve (AUPRC): {auprc:.2f}')
        
        
    def xg_boost_classification(self, X_train_embeddings,y_train,X_test_embeddings,y_test):
        print("Doing classification using XG Boost...")
        
        clf = xgb.XGBClassifier(scale_pos_weight=200, max_depth=40, learning_rate=0.2, n_estimators=100, gamma=0.3)
        clf.fit(X_train_embeddings, y_train)

        probas = clf.predict_proba(X_test_embeddings)  # Get probabilities for the positive class
        y_scores = probas[:, 1]  # Probabilities for class 1
    
        # Example: Compute precision-recall curve
        precision, recall, _ = precision_recall_curve(y_test, y_scores)
        
        # Example: Compute AUC (Area Under Curve) for precision-recall curve
        auprc = auc(recall, precision)
        
        print(f'Area Under Precision-Recall Curve (AUPRC): {auprc:.2f}')

        print("Results of XG Boost are: ")
        print(classification_report(y_test, y_pred))
        
    def random_forest(self, X_train_embeddings,y_train,X_test_embeddings,y_test):
        print("Doing classification using Random Forest...")
        

        # Instantiate the Random Forest Classifier
        clf = RandomForestClassifier(n_estimators=100, max_depth=10, max_leaf_nodes= 500, n_jobs= 2, max_features="sqrt", class_weight={0:1, 1:30}, verbose=True)
        clf.fit(X_train_embeddings, y_train)

        # Predict and Evaluate
        y_pred = clf.predict(X_test_embeddings)
        print("Results of Random Forest are: ")
        print(classification_report(y_test, y_pred))
        probas = clf.predict_proba(X_test_embeddings)  # Get probabilities for the positive class
        y_scores = probas[:, 1]  # Probabilities for class 1
    
        # Example: Compute precision-recall curve
        precision, recall, _ = precision_recall_curve(y_test, y_scores)
        
        # Example: Compute AUC (Area Under Curve) for precision-recall curve
        auprc = auc(recall, precision)
        
        print(f'Area Under Precision-Recall Curve (AUPRC): {auprc:.2f}')
        
    def cat_boost(self, X_train_embeddings,y_train,X_test_embeddings,y_test):
        print("Doing classification using Cat Boost...")
        
        clf = CatBoostClassifier(iterations=500, 
                         depth=10, 
                         learning_rate=0.1, 
                         custom_loss='Logloss',
                         verbose=200,
                         class_weights=[1, 30])

        clf.fit(X_train_embeddings, y_train)

        # Predict and Evaluate
        y_pred = clf.predict(X_test_embeddings)
        print("Results of Cat Boost classifier are: ")
        print(classification_report(y_test, y_pred))
        
    def interpretation_call(self):

        causal_test_df=self.test_df[self.test_df["label"]==1]
        causal_df_unique = causal_test_df.drop_duplicates(subset=['index', 'id1', 'id2'])

        samp_abst=causal_df_unique[causal_df_unique["index"]==25064704]

        # Load the model from a file
        clf = load('CRED_trained_model.joblib')

        model_name = "dmis-lab/biobert-base-cased-v1.1"
        biobert = BertModel.from_pretrained(model_name).to(self.device)
        tokenizer = BertTokenizer.from_pretrained(model_name)

        for _, row in tqdm(causal_df_unique.iterrows()):
            ranked, word_importance = interpretation(row, clf, tokenizer, biobert, self.get_specific_token_embeddings)

    
    
  
mod_tr = model_trainer(train_data, test_data, "dmis-lab/biobert-base-cased-v1.1")

#To generate embeddings uncomment the below line
X_train_embeddings,X_test_embeddings= mod_tr.generate_embeddings()


X_train,y_train,X_test,y_test = mod_tr.create_dataset()

X_train_embeddings,X_test_embeddings,y_train,y_test = mod_tr.drop_null_embeddings(X_train_embeddings,X_test_embeddings,y_train,y_test)
print("Embeddings generated")


/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BioBERT model loaded
Tokenizer loaded
Dataset created


 61%|█████████████████████████████████████████████████▊                               | 2791/4541 [00:38<00:28, 60.47it/s]

Salidroside Attenuates Cognitive Dysfunction in Senescence-Accelerated Mouse Prone 8 ( SAMP8 ) Mice and Modulates Inflammation of the Gut-Brain Axis . Background : Alzheimer 's disease ( AD ) is a fatal neurodegenerative disease characterized by progressive cognitive decline and memory loss . However , several therapeutic approaches have shown unsatisfactory outcomes in the clinical setting . Thus , developing alternative therapies for the prevention and treatment of AD is critical . Salidroside ( SAL ) is critical , an herb-derived phenylpropanoid glycoside compound , has been shown to attenuate lipopolysaccharide (LPS)-induced cognitive impairment . However , the mechanism underlying its neuroprotective effects remains unclear . Here , we show that SAL has a therapeutic effect in the senescence-accelerated mouse prone 8 ( SAMP8 ) strain , a reliable and stable mouse model of AD . Methods : SAMP8 mice were treated with SAL , donepezil ( DNP ) or saline , and cognitive behavioral impai

 64%|███████████████████████████████████████████████████▊                             | 2906/4541 [00:40<00:26, 61.37it/s]

Systemic infection exacerbates cerebrovascular dysfunction in Alzheimer 's disease . We studied the effects of systemic infection on brain cytokine level and cerebral vascular function in Alzheimer 's disease and vascular dementia , in superior temporal cortex ( Brodmann area 22 ) from Alzheimer 's disease patients ( n = 75 ) , vascular dementia patients ( n = 22 ) and age-matched control subjects ( n = 46 ) , stratified according to the presence or absence of terminal systemic infection . Brain cytokine levels were measured using Mesoscale Discovery Multiplex Assays and markers of cerebrovascular function were assessed by ELISA . Multiple brain cytokines were elevated in Alzheimer 's disease and vascular dementia : IL-15 and IL-17A were maximally elevated in end-stage Alzheimer 's disease ( Braak tangle stage V-VI ) whereas IL-2 , IL-5 , IL12p40 and IL-16 were highest in intermediate Braak tangle stage III-IV disease . Several cytokines ( IL-1beta , IL-6 , TNF-alpha , IL-8 and IL-15 )

100%|███████████████████████████████████████████████████████████████████████████████████| 615/615 [00:08<00:00, 72.64it/s]

Embeddings generated
Dataset created
Embeddings generated


# SVM

In [7]:

def svm_classification(X_train_embeddings,y_train,X_test_embeddings,y_test):
    print("Doing classification using SVM...")
    clf = SVC(kernel='poly', degree=10, probability=True, class_weight={0:1, 1:30}, gamma='scale', C=10)
    clf.fit(X_train_embeddings, y_train)
    dump(clf, 'svm_model.joblib')
    print('model saved')


    probas = clf.predict_proba(X_test_embeddings)  # Get probabilities for the positive class
    y_scores = probas[:, 1]  # Probabilities for class 1

    # Example: Compute precision-recall curve
    precision, recall, _ = precision_recall_curve(y_test, y_scores)
    
    # Example: Compute AUC (Area Under Curve) for precision-recall curve
    auprc = auc(recall, precision)
    
    print(f'Area Under Precision-Recall Curve (AUPRC): {auprc:.2f}')

    return probas, clf

In [8]:
proba, clf = svm_classification(X_train_embeddings,y_train,X_test_embeddings,y_test)

Doing classification using SVM...
model saved
Area Under Precision-Recall Curve (AUPRC): 0.47


# XG-boost

In [27]:
def xg_boost_classification( X_train_embeddings,y_train,X_test_embeddings,y_test):
    print("Doing classification using XG Boost...")
        
    clf = xgb.XGBClassifier(scale_pos_weight=500, max_depth=40, learning_rate=0.1, n_estimators=400, gamma=0.1)
    clf.fit(X_train_embeddings, y_train)

    # Get probabilities
    probas = clf.predict_proba(X_test_embeddings)
    y_scores = probas[:, 1]  # Probabilities for class 1

    # Example: Compute precision-recall curve
    precision, recall, _ = precision_recall_curve(y_test, y_scores)
    
    # Example: Compute AUC (Area Under Curve) for precision-recall curve
    auprc = auc(recall, precision)
    
    print(f'Area Under Precision-Recall Curve (AUPRC): {auprc:.2f}')

    return probas, clf


In [28]:
proba, clf = xg_boost_classification(X_train_embeddings,y_train,X_test_embeddings,y_test)

Doing classification using XG Boost...
Area Under Precision-Recall Curve (AUPRC): 0.38


# Random Forest

In [29]:

def random_forest(X_train_embeddings,y_train,X_test_embeddings,y_test):



    # Instantiate the Random Forest Classifier
    clf = RandomForestClassifier(n_estimators=200,n_jobs=5, max_depth=50, max_leaf_nodes= 500,  max_features="sqrt", class_weight={0:1, 1:30})
    clf.fit(X_train_embeddings, y_train)

    probas = clf.predict_proba(X_test_embeddings)
    y_scores = probas[:, 1]  # Probabilities for class 1

    # Example: Compute precision-recall curve
    precision, recall, _ = precision_recall_curve(y_test, y_scores)
    
    # Example: Compute AUC (Area Under Curve) for precision-recall curve
    auprc = auc(recall, precision)
    
    print(f'Area Under Precision-Recall Curve (AUPRC): {auprc:.2f}')

    return probas, clf

In [30]:
proba, clf = random_forest(X_train_embeddings,y_train,X_test_embeddings,y_test)

Area Under Precision-Recall Curve (AUPRC): 0.41


# Pubmedbert

In [17]:
pubmed_bert = pd.read_csv("pubmedbert_probs_new_data.csv")
# Calculate the average precision score
y_scores_pub_bert = pubmed_bert["1"]
precision_pub_bert, recall_pub_bert, _ = precision_recall_curve(y_test, y_scores_pub_bert)
auprc_pub_bert = auc(recall_pub_bert, precision_pub_bert)

print(f'Area Under Precision-Recall Curve (AUPRC): {auprc_pub_bert:.2f}')


Area Under Precision-Recall Curve (AUPRC): 0.42


# Scibert

In [18]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
scibert = pd.read_csv("scibert_probs_new_data.csv")


# Calculate the average precision score
y_scores_scibert = scibert["1"]
precision_scibert, recall_scibert, _ = precision_recall_curve(y_test, y_scores_scibert)
auprc_scibert = auc(recall_scibert, precision_scibert)

print(f'Area Under Precision-Recall Curve (AUPRC): {auprc_scibert:.2f}')


Area Under Precision-Recall Curve (AUPRC): 0.46


# Biobert

In [26]:

# Set the seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
set_seed(seed)

# Define the compute metrics function for evaluation
def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=1)
    return {'accuracy': (preds == labels).mean()}

# Load datasets
test_df = pd.read_csv("test_df full.csv")
train_df = pd.read_csv("new_train_df (1).csv")
test_df.drop(columns = ["Unnamed: 0"], inplace=True)

train_df = train_df[["sentence", "label"]]
test_df = test_df[["sentence", "label"]]


# Convert your dataframes to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


# Tokenize your dataset
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


# Set format for pytorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


# Define training arguments
training_args = TrainingArguments(
    output_dir='./bioresults',            # output directory
    num_train_epochs=4,                   # total number of training epochs
    learning_rate=3e-5,                   # learning rate
    per_device_train_batch_size=8,        # batch size per device during training
    per_device_eval_batch_size=4,         # batch size for evaluation
    warmup_steps=1000,                       # number of warmup steps for learning rate scheduler
    weight_decay=0.05,                    # strength of weight decay
    logging_dir='./biologs',              # directory for storing logs
    load_best_model_at_end=True,          # load the best model at the end of training
    metric_for_best_model="accuracy",     # use accuracy for best model
    evaluation_strategy="epoch",          # evaluate each epoch
    save_strategy="epoch",                # save checkpoints each epoch
)

# Load model
model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1", num_labels=2)

# Save the model with tensors made contiguous
def save_model_with_contiguous(model, output_dir):
    # Ensure all tensors are contiguous before saving
    for param in model.parameters():
        param.data = param.data.contiguous()
    
    model.save_pretrained(output_dir)

# Modify the Trainer to include the new saving function
class CustomTrainer(Trainer):
    def _save_checkpoint(self, model, trial, metrics=None):
        output_dir = self._get_output_dir(trial=trial)
        self.save_model(output_dir)

    def save_model(self, output_dir, _internal_call=True):
        save_model_with_contiguous(self.model, output_dir)

# Use CustomTrainer instead of Trainer
trainer = CustomTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics=compute_metrics      # metrics function for evaluation
)

# Train the model
trainer.train()

# Evaluate the model
predictions = trainer.predict(test_dataset)
probabilities = softmax(predictions.predictions, axis=1)

y_pred = np.argmax(probabilities, axis=1)

report = classification_report(y_test, y_pred, output_dict=True)

# Print the classification report
print(report)

# Calculate the average precision score
y_scores_bluebert = probabilities[:, 1]
precision_bluebert, recall_bluebert, _ = precision_recall_curve(y_test, y_scores_bluebert)
auprc_bluebert = auc(recall_bluebert, precision_bluebert)

print(f'Area Under Precision-Recall Curve (AUPRC): {auprc_bluebert:.2f}')




/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/4541 [00:00<?, ? examples/s]

Map:   0%|          | 0/615 [00:00<?, ? examples/s]

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.332800,0.334235,0.923577
2,0.214400,0.326716,0.913821
3,0.176300,0.448266,0.921951
4,0.118800,0.469814,0.928455


{'0': {'precision': 0.9495652173913044, 'recall': 0.9732620320855615, 'f1-score': 0.9612676056338029, 'support': 561.0}, '1': {'precision': 0.625, 'recall': 0.46296296296296297, 'f1-score': 0.5319148936170213, 'support': 54.0}, 'accuracy': 0.9284552845528455, 'macro avg': {'precision': 0.7872826086956521, 'recall': 0.7181124975242622, 'f1-score': 0.7465912496254121, 'support': 615.0}, 'weighted avg': {'precision': 0.9210668080593849, 'recall': 0.9284552845528455, 'f1-score': 0.9235683431152562, 'support': 615.0}}
Area Under Precision-Recall Curve (AUPRC): 0.43


SVM: 0.47 <br>
XG Boost: 0.38 <br>
Random Forest: 0.41 <br>
Pubmedbert: 0.42 <br>
Scibert: 0.46 <br>
Biobert: 0.43

# Ends